<a href="https://colab.research.google.com/github/YuanYuan-11111101001/CyberVSR-2023/blob/main/Feature_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns
import scipy.stats as ss
# from scipy.stats import entropy

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# converting timestamps to week number of the year. Remember to load dataset first
date = pd.to_datetime(august['timestamp'], errors='coerce')

date.astype('int64').dtypes

weeknumbers = date.dt.week

august['week'] = weeknumbers
# week number 5-9 for February
# 33 - 35 for August

<ipython-input-7-9331564bd20e>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  weeknumbers = date.dt.week


In [ ]:
# Example: separating dataset by week number
week_33 = august[august['week'] == 33]
week_34 = august[august['week'] == 34]
week_35 = august[august['week'] == 35]

In [ ]:
df = week_35

# Functions

## For username, password, and command:

In [ ]:
def num_valid(df, columns):
  '''
  Returns the number of valid records in a dataset where a value is not considered
  valid iff it is empty/NaN CHANGE to return serieSSS
  '''
  valid_numbers = {}
  for column in columns:
    x = df.dropna(subset = [column])
    valid_numbers[column] = len(x)

  return pd.Series(valid_numbers)

In [ ]:
def percent_valid(df, columns):
  '''
  returns percent of non-empty records in each column in columns
  '''
  percent_valid = {}
  for column in columns:
    percent_valid[column] = len(df.dropna(subset = [column]))/len(df)

  return pd.Series(percent_valid)

In [ ]:
def unique_valid(df, columns):
  '''
  returns number of unique non-empty records in each column in columns
  '''
  valid_unique = {}
  for column in columns:
    valid_unique[column] = df[column].nunique(dropna=True)
  return pd.Series(valid_unique)

## For Src_port, Dest_port, and sensor

In [ ]:
def find_outliers(data, column, include_NaN=False):
  '''
  Identify outliers within a categorical field of a dataframe. Returns dataframe with outlier values.
  Returns an empty dataframe if there are none.

  An outlier is defined as a a value that falls outside three standard deviations of the mean.
  We convert the categorical variables into the frequencies in which each unique value occurs
  within that field. We define the mean as the mean frequency of occurrence and the standard
  deviation as the standard deviation of frequencies. Values with frequencies that are not
  within three standard deviations of the mean frequency will be returned in the dataframe
  '''
  frequencies = data[column].value_counts(normalize=True,dropna=include_NaN)

  data['freq'] = frequencies

  mean = frequencies.mean()
  std = frequencies.std()

  return data[(data['freq'] <= mean-3*std) | (data['freq'] >= mean+3*std)]

In [ ]:
def is_outlier(data, column, outliers):

  counter = 0
  total = 0
  for value in data[column]:
    total += 1
    if value in outliers:
      counter += 1
  # print(len(df))
  # len(df) not working????
  return counter/total

In [ ]:
def most_common(data, column):
  '''
  Return most common unique value in column
  '''
  # for column in columns:
  #   most = data[column].value_counts(dropna=True).nlargest(3)
  #   most['column_name'] = column
  top_three = data[column].value_counts(dropna=True)[:3].index.tolist()


  return top_three

In [ ]:
def is_common(data, column, common):
  counter = 0
  for value in data[column]:
    if value in common:
      counter += 1

  return counter


In [ ]:
def entropy(df, columns, same_base=True):
  '''
  return entropy values of each column
  '''
  entropies = {}
  for col in columns:
      freq = df[col].value_counts(normalize=True,dropna=True)
      if same_base:
          # ent = -freq * np.log2(freq)
          ent = freq * np.log2(freq)
      else:
          base =  df[col].nunique(dropna=False)
          # ent = -freq * np.log2(freq)/np.log2(base)
          ent = freq * np.log2(freq)/np.log2(base)
      ent = -1 * ent.sum()
      entropies[col] = ent
  return pd.Series(entropies)

In [ ]:
# Calculating cross entropies. Note that there are several versions of cross entropy to accomodate different RAM limits
# To accomodate for google colab free version limited RAM, used get_p, qstep_1, and ce_steps
def cross_entropy(df1, df2, columns):
  '''
  Where df1 is the true and df2 is pred.

  Don't include NaN for fields
  '''

  freq_true = df1[columns].value_counts(normalize=True,dropna=False)
  ent = freq_true * np.log2(df2)
  ent = -1 * ent.sum()
  return ent

def cross_entropy_m(df1, df2, columns):
  '''
  Where df1 is the true and df2 is pred.

  Don't include NaN for fields
  '''
  df2 = df1.copy()
  df2 = df2[df2['src_ip'] != df1['src_ip']]
  df2 = df2[columns].value_counts(normalize=True, dropna=False)

  freq_true = df1[columns].value_counts(normalize=True,dropna=False)
  ent = freq_true * np.log2(df2)

  ent = -1 * ent.sum()
  return ent

def cross_entropy_combined(df1, df2):
  df = df2.copy()
  ent = df1*np.log2(df2)
  ent = -1 * ent.sum()
  df['ce'] = ent
  return ent

def get_p(df, column):
  # Returns p distribution for calculating cross entropy
  distribution = df[column].value_counts(normalize=True,dropna=True)
  if distribution.empty:
    return pd.Series({np.nan: 0})
  else:
    return distribution

def get_q(df, dff, column):
  q = dff[~dff['src_ip'].isin(df['src_ip'])]
  return q[column].value_counts(normalize=True,dropna=True).tolist()

def get_q1(df, dff, column):
  return dff[~dff['src_ip'].isin(df['src_ip'])]

def get_q2(df, column):
  return df[column].value_counts(normalize=True,dropna=True)

def get_qstep1(df, df2, column):
  '''
  Returns normalized (divide by total size) df2 minus the value counts of df[column].
  where df is the dataframe and df2 is a value_counts list of distribution q.
  '''
  df_ret = df2.copy()
  values = df[column].value_counts(dropna = True)
  for value in values.keys():
    df_ret.loc[value] = df2.loc[value] -values.loc[value]
  return df2/(df2.sum())

def ce_steps(df1, df2, name_of_ce):
  '''
  returns dataframe of each source ip and it's cross entropy. where df1 is p
  and df2 is q.
  '''
  c = []
  ind = []
  for i in df2.index:
    ent = df1.loc[i]*np.log2(df2.loc[i].dropna())
    ent = -1*ent.sum()

    ind.append(i)
    c.append(ent)

  data = {'src_ip':ind, 'ce ' + name_of_ce: c}

  ces = pd.DataFrame(data)
  ces = ces.set_index('src_ip')

  return ces

def cross_entropy_base(df1, df2, columns):
  freq_true = df1[columns].value_counts(normalize=True,dropna=False)

  # ent = -freq_true * np.log2(df2)
  base =  len(df2)
  ent = -freq_true * np.log2(df2)/np.log2(base)

  ent = ent.sum()
  return ent

# Feature Transformation:

In [ ]:
unique_src = df.groupby('src_ip').size()
attack_freq = pd.DataFrame(unique_src)
attack_freq.columns=['attack_frequency']
attack_freq

## Calculate entropies

In [ ]:
entropy_columns = ['src_port', 'dest_port', 'sensor', 'username', 'ssh_username', 'command']

In [ ]:
entropy_dataset = df.groupby('src_ip')[entropy_columns].apply(entropy, columns=entropy_columns)
new_col_names = ['entropy '+ i for i in entropy_dataset.columns]
entropy_dataset.columns = new_col_names
# Print the results
entropy_dataset

## Group One: username, ssh_username, command

In [ ]:
group_one = ['username', 'ssh_username', 'command']

In [ ]:
number_valid_df = df.groupby(['src_ip'])[group_one].apply(num_valid, columns = group_one)

new_col_names_nv = ['num_valid '+ i for i in number_valid_df.columns]
number_valid_df.columns = new_col_names_nv

In [ ]:
percent_valid_df = df.groupby(['src_ip'])[group_one].apply(percent_valid, columns = group_one)

new_col_names_pv = ['percent_valid '+ i for i in percent_valid_df.columns]
percent_valid_df.columns = new_col_names_pv

In [ ]:
unique_valid_df = df.groupby(['src_ip'])[group_one].apply(unique_valid, columns = group_one)

new_col_names_uv = ['unique_valid '+ i for i in unique_valid_df.columns]
unique_valid_df.columns = new_col_names_uv

## Group Two: src_port, dest_port, sensor

In [ ]:
group_two = ['src_port', 'dest_port', 'sensor']

In [ ]:
# Find Outlier Lists
src_outliers = find_outliers(df, 'src_port', True)
dest_outliers = find_outliers(df, 'dest_port', True)
sensor_outliers = find_outliers(df, 'sensor', True)

src_outliers = src_outliers[['src_port']]
dest_outliers = dest_outliers[['dest_port']]
sensor_outliers = sensor_outliers[['sensor']]

<ipython-input-17-347a07de33c9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['freq'] = frequencies
<ipython-input-17-347a07de33c9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['freq'] = frequencies
<ipython-input-17-347a07de33c9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [ ]:
df['src_port'].nunique()

7375

In [ ]:
outliers = pd.DataFrame()

In [ ]:
outliers['src_port'] = df.groupby(['src_ip'])[['src_port']].apply(is_outlier, 'src_port', src_outliers['src_port'])
outliers['dest_port'] = df.groupby(['src_ip'])[['dest_port']].apply(is_outlier, 'dest_port', dest_outliers['dest_port'])
outliers['sensor'] = df.groupby(['src_ip'])[['sensor']].apply(is_outlier, 'sensor', sensor_outliers['sensor'])

In [ ]:
new_col_names_o = ['num_outliers '+ i for i in outliers.columns]
outliers.columns = new_col_names_o

In [ ]:
# Find top 3 most common values:

# df[['src_port']].value_counts(dropna=True)[:3].index.tolist()
src_common =  most_common(df, ['src_port'])
dest_common =  most_common(df, ['dest_port'])
sensor_common =  most_common(df, ['sensor'])

In [ ]:
num_common = pd.DataFrame()

num_common['num_common src_port'] = df.groupby(['src_ip'])[['src_port']].apply(is_common, 'src_port', [40601,47001,54201])
num_common['num_common dest_port'] = df.groupby(['src_ip'])[['dest_port']].apply(is_common, 'dest_port', [23, 445, 2222])
num_common['num_common sensor'] = df.groupby(['src_ip'])[['sensor']].apply(is_common, 'sensor', ['db542e60-bcd5-422e-b627-c28f895858ea','40a6483a-70d1-42ab-88c7-c63eb7d4cea2','1d911ad4-0dac-4200-9abd-92ce03e7bf59'])

# Cross Entropy

In [ ]:
# get p distribution for calculating cross entropy
psp = df.groupby('src_ip').apply(get_p, column = 'src_port')
pdp = df.groupby('src_ip').apply(get_p, column = 'dest_port')
pu = df.groupby('src_ip').apply(get_p, column = 'username')
psshu = df.groupby('src_ip').apply(get_p, column = 'ssh_username')
pc = df.groupby('src_ip').apply(get_p, column = 'command')

In [ ]:
comparative_sp = df['src_port'].value_counts(dropna = True)

df_qsp = df.groupby('src_ip').apply(get_qstep1, df2 = comparative_sp, column = 'src_port')
df_qsp = df_qsp.replace(0,np.nan)

In [ ]:
comparative_dp = df['dest_port'].value_counts(dropna = True)

df_qdp = df.groupby('src_ip').apply(get_qstep1, df2 = comparative_dp, column = 'dest_port')
df_qdp = df_qdp.replace(0,np.nan)

In [ ]:
comparative_u = df['username'].value_counts(dropna = True)

df_qu = df.groupby('src_ip').apply(get_qstep1, df2 = comparative_u, column = 'username')
df_qu = df_qu.replace(0,np.nan)

In [ ]:
comparative_sshu = df['ssh_username'].value_counts(dropna = True)

df_qsshu = df.groupby('src_ip').apply(get_qstep1, df2 = comparative_sshu, column = 'ssh_username')
df_qsshu = df_qsshu.replace(0,np.nan)

In [ ]:
comparative_c = df['command'].value_counts(dropna = True)

df_qc = df.groupby('src_ip').apply(get_qstep1, df2 = comparative_c, column = 'command')
df_qc = df_qc.replace(0,np.nan)

In [ ]:
df_ce_sp = ce_steps(psp, df_qsp, 'src_port')
df_ce_dp = ce_steps(pdp, df_qdp, 'dest_port')
df_ce_u = ce_steps(pu, df_qu, 'username')
df_ce_sshu = ce_steps(psshu, df_qsshu, 'ssh_username')
df_ce_c = ce_steps(pc, df_qc, 'command')

In [ ]:
cross_entropy_df = pd.concat([df_ce_sp, df_ce_dp, df_ce_u, df_ce_sshu, df_ce_c], axis=1)

In [ ]:
cross_entropy_df.columns = ['ce src_port', 'ce dest_port', 'ce username', 'ce ssh_username', 'ce command']

In [ ]:
cross_columns = ['src_port', 'dest_port', 'sensor', 'username', 'ssh_username', 'command']

# Create final df

In [ ]:
# transformed features of the dataframe are used in unusual ips for processing and analysis.
final_df_names = [attack_freq, entropy_dataset, number_valid_df, percent_valid_df, unique_valid_df, outliers, num_common, cross_entropy_df]

In [ ]:
result = pd.concat(final_df_names, axis=1)